In [1]:
import folium #tiene varios conjuntos de mosaicos integrados de OpenStreetMap
import h3 #indexación geoespacial jerárquico hexagonal H3 de Uber 
from shapely.geometry import Polygon, shape  #para la manipulación y el análisis de objetos geométricos planos
import pandas as pd
import shapely
import numpy as np
import asyncio                
import nest_asyncio
nest_asyncio.apply()
import twint #scraping en Twitter

In [2]:
#Cargamos el archivo geojson 
geojson = pd.read_json('area_ecuador.geojson')

In [3]:
#Agregue un búfer de 1 grado alrededor de esta área
s = shape(geojson['features'][0]['geometry']) # obtiene # en f y c de una matriz (shape) de las coordenadas 
s = s.buffer(0.2) #mas cercania linea verde
feature = {'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(s)}
feature['geometry']['coordinates'] = [[[v[0], v[1]] for v in feature['geometry']['coordinates'][0]]]
feature = feature['geometry']
feature['coordinates'][0] = [[v[1], v[0]] for v in feature['coordinates'][0]]

In [4]:
# encontrar todos los hexágonos con un centro que se encuentra dentro de nuestra 
    # área de interés amortiguada desde arriba
polyline = feature['coordinates'][0]
polyline.append(polyline[0])
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]
#print(lat)
m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=6, tiles='cartodbpositron')
my_PolyLine=folium.PolyLine(locations=polyline,weight=4,color="green")
m.add_child(my_PolyLine)

#se crea la lista de IDs de heganosos en base a las caracteristicas de las coordenadas
hexagons = list(h3.polyfill(feature, 4)) 

# mapeo de los hexagonos
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        
    # aplanar polígonos en bucles
    # convertir varias colecciones del formato geojson de las coordenadas en una sola
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    
    lat.extend(map(lambda v:v[0],polyline))
    lng.extend(map(lambda v:v[1],polyline))
    polylines.append(polyline)
for polyline in polylines:
    my_PolyLine=folium.PolyLine(locations=polyline,weight=4,color='blue')
    m.add_child(my_PolyLine)
display(m)

In [6]:
#se muestra el total de hexagonos que cubre el territorio de Ecuador
len(hexagons)

130

In [6]:
#(h3.cell_area(hexagons[0], unit='km^2'))/np.pi
np.sqrt((h3.cell_area(hexagons[0], unit='km^2')/0.827)/np.pi)

28.30857975856057

# Extracción de tweets depresivos

In [7]:
palabras_clave=["agobiado","agobiada","angustiado","angustiada",
"ansiedad","decaído","depresión","depresivo", "depresiva","deprimido",
"deprimida","desanimado","desanimada","desesperado","desesperada",
"desmotivado","desmotivada","nervioso", "antidepresivo",
"antidepresivos","Desesperanzado","suicidio"]
#clave=["depresion"]

In [9]:
c = twint.Config() 
def buscar_por_ciudad(since, until, outfile):
    for hex in hexagons:
        center = h3.h3_to_geo(hex)  # almacena las coordenasas de centro de cada hexagono
         
        r = np.sqrt((h3.cell_area(hex, unit='km^2')/0.827)/np.pi) #calcula el radio de cada hexag 

        #str(center[0]) + "," + str(center[1]) + "," + str(r) + "km"
        
        for keyword in palabras_clave:
            #print(city)
            c.Search = keyword
            c.Geo = str(center[0]) + "," + str(center[1]) + "," + str(r) + "km" #lat, lon, radio
            c.Since = since
            c.Until = until
            c.Links = "exclude" #excluir tweets que contengan links
            #c.Store_object = True
            c.Filter_retweets = True # Excluir retweets de los resultados. 
            #c.Limit = 10 
            c.Store_csv = True
            #follow_df = twint.storage.panda.User_df   #DATAFRAME
            #c.Custom = ["date", "time", "username", "tweet"]
            c.Output = "./" + outfile #"ambato1.csv"
            c.Hide_output = True     # para que la salida no se uestre en la consola
            try:
                twint.run.Search(c)
            except:
                print('error en leer esta celda')
            
buscar_por_ciudad('2020-01-30  00:00:00', '2021-12-31  00:00:00', 'Dataset_inicial.csv')
#print("hola")
#print("Centro:", centro)
#print("Radio:", radio)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

...

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [26]:
#leemos el dataset extraido para poder visualizar los datos extraidos de los tweets depresivos
df = pd.read_csv('Dataset_inicial.csv')
#df.to_excel('Dataset_inicial.xlsx', index=False)
df.head(5)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1284680153663246338,1284628509982363651,"2020-07-18 21:43:11 Hora est. Pacífico, Sudamé...",2020-07-18,21:43:11,-500,250476526,xavierconequiz,Xavi Chuncha Simbaña,NaN,...,"-1.3695361279507605,-78.08139447223985,27.9857...",NaN,NaN,NaN,NaN,"[{'screen_name': 'fabovillamar', 'name': 'Fabr...",NaN,NaN,NaN,NaN
1,1339912665586610177,1339912665586610177,"2020-12-18 07:37:28 Hora est. Pacífico, Sudamé...",2020-12-18,07:37:28,-500,1558953666,johasolano27,Joha Solano♥️,NaN,...,"-1.3695361279507605,-78.08139447223985,27.9857...",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1369099577991847937,1369099577991847937,"2021-03-08 20:35:50 Hora est. Pacífico, Sudamé...",2021-03-08,20:35:50,-500,64263405,estebanacostab,esteban acosta,NaN,...,"-1.3695361279507605,-78.08139447223985,27.9857...",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1401985494934863874,1401985494934863874,"2021-06-07 14:32:44 Hora est. Pacífico, Sudamé...",2021-06-07,14:32:44,-500,1242893257555824640,librasequilibra,Mel ♎️,NaN,...,"-1.3695361279507605,-78.08139447223985,27.9857...",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1396149060634087428,1396149060634087428,"2021-05-22 12:00:49 Hora est. Pacífico, Sudamé...",2021-05-22,12:00:49,-500,1242893257555824640,librasequilibra,Mel ♎️,NaN,...,"-1.3695361279507605,-78.08139447223985,27.9857...",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


En la presentación del dataframe anterior se puede observar algunos de los atributos que estan presentes en el dataset inicial, se puede observar que hay bastantes atributos que son innecesarios, esto se corrige en la fase de preprocesamiento.

# Extracción de tweets aleatorios

In [5]:
palabras_clave_random=["soy","estoy"]
c1 = twint.Config() 
def buscar_tweets_aleatorios(since, until, outfile):
        
    for keyword in palabras_clave_random:
        #print(city)
        c1.Search = keyword
       # "-1.2743089918452106, -78.70605468750001, 236km"
        c1.Geo = "-1.2743089918452106,-78.70605468750001,236km" #str(center[0]) + "," + str(center[1]) + "," + str(r) + "km" #lat, lon, radio
        c1.Since = since
        c1.Until = until
        c1.Limit = 5000
        c1.Links = "exclude" #excluir tweets que contengan links
        #c.Store_object = True
        c1.Filter_retweets = True # Excluir retweets de los resultados. 
        c1.Store_csv = True
        c1.Output = "./" + outfile #"ambato1.csv"
        c1.Hide_output = True     # para que la salida no se uestre en la consola
        try:
            twint.run.Search(c1)
        except:
            print('error en leer esta celda')
            
buscar_tweets_aleatorios('2020-01-30  00:00:00', '2021-12-31  00:00:00', 'Dataset_random.csv')